In [47]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

In [48]:
base = pd.read_csv('games.csv')
base.head(5)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Developer', axis = 1)
base = base.drop('NA_Sales', axis = 1)
base = base.drop('EU_Sales', axis = 1)
base = base.drop('JP_Sales', axis = 1)

In [50]:
base = base.dropna(axis = 0) # remove all rows containing NaN values

In [51]:
base = base.loc[base['Global_Sales'] > 1]

In [52]:
base.shape

(1307, 11)

In [53]:
base['Name'].value_counts()

FIFA Soccer 08                                 5
The Simpsons Game                              5
Battlefield 4                                  5
LEGO Indiana Jones: The Original Adventures    5
Star Wars: The Force Unleashed                 5
                                              ..
WWE 2K16                                       1
World of Warcraft: Mists of Pandaria           1
Wii Sports Resort                              1
Halo: The Master Chief Collection              1
Need For Speed: High Stakes                    1
Name: Name, Length: 996, dtype: int64

In [54]:
base = base.drop('Name', axis = 1)
base.head(4)

,Platform,Year_of_Release,Genre,Publisher,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,82.53,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,35.52,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,32.77,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,29.80,89.0,65.0,8.5,431.0,E


In [55]:
y =  base['Global_Sales']
base = base.drop('Global_Sales', axis = 1)

In [56]:
base.head(4)

,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,76.0,51.0,8,322.0,E
2,Wii,2008.0,Racing,Nintendo,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,80.0,73.0,8,192.0,E
6,DS,2006.0,Platform,Nintendo,89.0,65.0,8.5,431.0,E


In [57]:
y.head(4)

0    82.53
2    35.52
3    32.77
6    29.80
Name: Global_Sales, dtype: float64

In [58]:
previsores = base.iloc[:, : ].values
venda_glob = y.values

In [59]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
 
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,2,3,8])],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()

In [60]:
previsores.shape

(1307, 99)

In [67]:
from tensorflow.keras import activations

In [71]:
camada_entrada = Input(shape=(99,))

camada_oculta1 = Dense(units = 50, # (99+1)/2 = 50
                       activation='relu')(camada_entrada)

camada_oculta2 = Dense(units = 50,
                      activation='relu')(camada_oculta1)

camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)

regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1])
regressor.compile(optimizer = 'adam',
                  loss = 'mse')

regressor.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 99)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5000      
_________________________________________________________________
dense_6 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [73]:
camada_entrada = Input(shape=(99,))

camada_oculta1 = Dense(units = 50, # (99+1)/2 = 50
                       )(camada_entrada)

camada_oculta2 = Dense(units = 50)(activations.relu(x=camada_oculta1, alpha=0.0, max_value=None, threshold=0))

camada_saida1 = Dense(units = 1, activation = 'linear')(activations.relu(x=camada_oculta2, alpha=0.0, max_value=None, threshold=0))

regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1])
regressor.compile(optimizer = 'adam',
                  loss = 'mse')

regressor.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 99)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5000      
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 50)]              0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
tf_op_layer_Relu_1 (TensorFl [(None, 50)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 51        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
____________________________________________________

In [74]:
regressor.fit(previsores, [venda_glob],
              epochs = 4000, batch_size = 100)

Epoch 1/4000
14/14 [==============================] - 0s 1ms/step - loss: 1633.8558
Epoch 2/4000
14/14 [==============================] - 0s 1ms/step - loss: 229.0896
Epoch 3/4000
14/14 [==============================] - 0s 1ms/step - loss: 163.7495
Epoch 4/4000
14/14 [==============================] - 0s 1ms/step - loss: 59.2655
Epoch 5/4000
14/14 [==============================] - 0s 1ms/step - loss: 38.9424
Epoch 6/4000
14/14 [==============================] - 0s 1ms/step - loss: 22.5919
Epoch 7/4000
14/14 [==============================] - 0s 1ms/step - loss: 17.9112
Epoch 8/4000
14/14 [==============================] - 0s 1ms/step - loss: 17.1517
Epoch 9/4000
14/14 [==============================] - 0s 1ms/step - loss: 19.1077
Epoch 10/4000
14/14 [==============================] - 0s 1ms/step - loss: 16.8487
Epoch 11/4000
14/14 [==============================] - 0s 1ms/step - loss: 17.2287
Epoch 12/4000
14/14 [==============================] - 0s 1ms/step - loss: 15.9737
Epoch 13/

In [72]:
regressor.fit(previsores, [venda_glob],
              epochs = 4000, batch_size = 100)

Epoch 1/4000
14/14 [==============================] - 0s 2ms/step - loss: 2949.8486
Epoch 2/4000
14/14 [==============================] - 0s 1ms/step - loss: 721.8010
Epoch 3/4000
14/14 [==============================] - 0s 1ms/step - loss: 219.9676
Epoch 4/4000
14/14 [==============================] - 0s 1ms/step - loss: 71.6968
Epoch 5/4000
14/14 [==============================] - 0s 1ms/step - loss: 28.5354
Epoch 6/4000
14/14 [==============================] - 0s 1ms/step - loss: 18.8919
Epoch 7/4000
14/14 [==============================] - 0s 1ms/step - loss: 18.7450
Epoch 8/4000
14/14 [==============================] - 0s 1ms/step - loss: 17.4284
Epoch 9/4000
14/14 [==============================] - 0s 1ms/step - loss: 16.3392
Epoch 10/4000
14/14 [==============================] - 0s 1ms/step - loss: 15.7207
Epoch 11/4000
14/14 [==============================] - 0s 1ms/step - loss: 15.5340
Epoch 12/4000
14/14 [==============================] - 0s 1ms/step - loss: 15.9853
Epoch 13/

In [75]:
previsao_glob = regressor.predict(previsores)

In [76]:
previsao_glob[0:10]

array([[81.63519  ],
       [36.24462  ],
       [35.063766 ],
       [16.687782 ],
       [10.880379 ],
       [25.836857 ],
       [25.00771  ],
       [18.59016  ],
       [ 3.4022908],
       [ 5.608289 ]], dtype=float32)

In [77]:
venda_glob[0:10]

array([82.53, 35.52, 32.77, 29.8 , 28.92, 28.32, 23.21, 22.7 , 21.81,
       21.79])